 # <h2><center><strong style="color:#ff6161">  Detecting Linux system hacking activities Part B
 </center></strong></h3>
 

<h2> 1. Producing the data 

<h3> 1.1 Memory Event Producer

In [ ]:
# Import libraries and initializing 

# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime 
from csv import DictReader


In [ ]:
def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [ ]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [ ]:
def add_timestamp(data_rows,tst):
    data = []
    #ts = datetime.datetime.utcnow().strftime("%s")
    for row in data_rows:
        
        row['ts'] = tst
        data.append({k: row[k] for k in row.keys()})
    return data_rows

In [ ]:
def alter_values(data_list):
    for sub in data_list:
        for key in sub:
            try:
                sub[key] = int(sub[key]) 
                sub['VGROW']= float(sub[key])
                sub['RGROW']= float(sub[key])
                sub['MEM']= float(sub[key])
                sub['VSIZE']= float(sub[key])
                sub['RSIZE']= float(sub[key])
            except ValueError:
                #sub[key] = float(sub[key])
                continue
    return data_list

In [ ]:
#Initializing the variables used in loop
ts = 0
machine_4 = []
machine_5 = []
machine_6 = []
machine_7 = []
machine_8 = []
combined_data =[]

#Initializing the producer instance
producer = connect_kafka_producer()

#Initializing the process topic name
topic = 'mem_pop_x4'

#Initializing the start index in the machine
start_index = {'machine_4' : 0,
               'machine_5' : 0,
               'machine_6' : 0,
               'machine_7' : 0,
               'machine_8' : 0}

#data_dict = {key : [data_list_X, data_list_Y, prev_Y]}
data_dict = {'machine_4' : [0,0,[]],
             'machine_5' : [0,0,[]],
             'machine_6' : [0,0,[]],
             'machine_7' : [0,0,[]],
             'machine_8' : [0,0,[]]}

# Reading the Linux process CSV file
input_file = DictReader(open('Streaming_Linux_memory.csv'))

# Seperating the records in by machines from the file
for row in input_file:
    #print(type(row['machine']))
    if int(row['machine']) == 4:
        machine_4.append(row)
    elif int(row['machine']) == 5:
        machine_5.append(row)
    elif int(row['machine']) == 6:
        machine_6.append(row)
    elif int(row['machine']) == 7:
        machine_7.append(row)
    else:
        machine_8.append(row)
        
#print(len(machine_8),len(machine_4),len(machine_5),len(machine_6),len(machine_7))

# Storing the machine records list for each machine key respectively
machine_list = {'machine_4' : machine_4,
                'machine_5' : machine_5,
                'machine_6' : machine_6,
                'machine_7' : machine_7,
                'machine_8' : machine_8}

print('Publishing records..')
while True:
    #Passing the machine and length of machine list
    for k, value in machine_list.items():
        #Checking for random rows to send
        rows_to_send_X =random.randint(20, 80)
        rows_to_send_Y =random.randint(0, 5) 
        
        #Setting the end index for X records 
        end_index_X = start_index[k] + rows_to_send_X
        #Setting the end index for Y records by adding the end_index of X and the rows to send for Y
        end_index_Y = end_index_X + rows_to_send_Y
        
        #Send the data with the enlisted rows
        data_dict[k][0] = value[start_index[k] : end_index_X]
        #Setting the timestamp
        ts = datetime.datetime.utcnow().strftime("%s")
        #Making changes to the list passed
        alter_values(data_dict[k][0]) 
        #Adding timestamp to the data list
        add_timestamp(data_dict[k][0],ts)
        #Setting the new end_index
        start_index[k] = end_index_X
        
        #Setting the start_index  as 0 if length of machine list is lesser than start_index
        if(start_index[k] >=len(value)):
            start_index[k]=0
        
        #Send the data with the enlisted rows
        data_dict[k][1] = value[start_index[k] : end_index_Y]
        #Making changes to the list passed
        alter_values(data_dict[k][1]) 
        #Adding timestamp to the data list
        add_timestamp(data_dict[k][1],ts)
        #Setting the new end_index
        start_index[k] = end_index_Y
        
        #Setting the start_index  as 0 if length of machine list is lesser than start_index
        if(start_index[k] >=len(value)):
            start_index[k]=0
            
    #Sending the data for all the machines in one list of sub-lists containing machine data                
    for key in data_dict.keys():
        #Checking if previous Y is not empty
        if(data_dict[key][2] != []):
            #Sends X and Y previous records together
            combined_data.append(data_dict[key][0] + data_dict[key][2])
        else:
            #Sends only X records
            combined_data.append(data_dict[key][0])
        
        #Setting the next Y records to the previous Y list for the next run
        data_dict[key][2] = data_dict[key][1]
        
    #Assigning sleep for 10 seconds between the published messages
    sleep(10)
    #Publish the combined data to topic
    publish_message(producer, topic, combined_data) 
    
    #Initialize the combined data list again to empty
    combined_data = []
    print('end cycle')